# そもそもどんなデータを扱うの？

ここでは，本サイトで分析対象とするデータについて簡単に紹介し，基礎分析を実施します．

なお，このデータは[MADB Labでv1.0として公開されているもの](https://github.com/mediaarts-db/dataset/tree/1.0)に[こちらの前処理](https://kakeami.github.io/viz-madb/appendix/preprocess.html)を実施したものです．ご興味のある方はAppendixをご参照ください．

## 環境構築

In [147]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [67]:
# 前処理の結果，以下に分析対象ファイルが格納されていることを想定
PATH_DATA = '../../data/preprocess/out/magazines.csv'

In [68]:
df = pd.read_csv(PATH_DATA)

## 概要

`df`は

- 週刊少年ジャンプ
- 週刊少年サンデー
- 週刊少年マガジン
- 週刊少年チャンピオン

の`1970-08-02`から`2017-07-06`までのすべての掲載作品のデータを格納した`DataFrame`です．
まずはサイズを見てみましょう．

In [69]:
df.shape

(179866, 12)

各週の掲載作品を一行ずつ格納しているため，合計で約18万行程度の規模になります．
以下に，各列の構成を示します．

In [70]:
df.columns

Index(['mcname', 'miname', 'cname', 'epname', 'creator', 'pageStart',
       'pageEnd', 'numberOfPages', 'datePublished', 'price', 'publisher',
       'editor'],
      dtype='object')

- `mcname`: 雑誌名
- `miname`: 雑誌巻号名
- `cname`: マンガ作品名
- `epname`: 各話タイトル
- `creator`: 作者名
- `pageStart`: 開始ページ
- `pageEnd`: 終了ページ
- `numberOfPages`: 雑誌の合計ページ数
- `datePublished`: 雑誌の発行日
- `price`: 雑誌の価格
- `publisher`: 雑誌の発行元
- `editor`: 雑誌の編集者（編集長）

冒頭数行を見て，データのイメージを掴んでみましょう．

In [71]:
df.head(3).T

,0,1,2
mcname,週刊少年サンデー,週刊少年マガジン,週刊少年サンデー
miname,週刊少年サンデー 1970年 表示号数32,週刊少年マガジン 1970年 表示号数32,週刊少年サンデー 1970年 表示号数32
cname,ぶッかれ*ダン,アシュラ,男どアホウ甲子園
epname,NaN,NaN,NaN
creator,赤塚不二夫,ジョージ秋山,水島新司と水島プロ
pageStart,3.0,19.0,33.0
pageEnd,32.0,66.0,50.0
numberOfPages,288.0,282.0,288.0
datePublished,1970-08-02,1970-08-02,1970-08-02
price,80.0,80.0,80.0


`pandas`の`describe()`コマンドでざっくり集計してみます．

In [72]:
df.describe()

,pageStart,pageEnd,numberOfPages,price
count,179866.000000,179860.000000,179559.000000,179828.000000
mean,210.909330,228.364308,417.366002,203.870732
std,126.082707,122.080065,67.403584,41.928839
min,1.000000,0.000000,36.000000,80.000000
25%,107.000000,126.000000,356.000000,180.000000
50%,205.000000,222.000000,437.000000,210.000000
75%,305.000000,322.000000,464.000000,236.000000
max,9999.000000,1578.000000,600.000000,371.000000


- `pageStart`はすべての行でデータがありますが，それ以外は`NaN`（数値なし）が結構ありそうです
- `numberOfPages`の最小値，最大値が想定外に広がっていたので，あとで深堀りします
- `price`も同様です．後で深堀りします

次に，`NaN`の数を列ごとに集計します．

In [148]:
df.isna().sum().reset_index()

,index,0
0,mcname,0
1,miname,0
2,cname,9
3,epname,26814
4,creator,441
5,pageStart,0
6,pageEnd,6
7,numberOfPages,307
8,datePublished,0
9,price,38


```{margin} 前処理が原因の欠測
`numberOfPages`，`price`に関しては，想定外のパターンのデータを`NaN`に変換するよう[前処理](https://kakeami.github.io/viz-madb/appendix/preprocess.html#cm102)を施しています．
```

特に`epname`と`publisher`の欠損が多いことがわかります．

## 列ごとの基礎分析

### `mcname`（雑誌名）

In [149]:
df['mcname'].value_counts().reset_index()

,index,mcname
0,週刊少年サンデー,46153
1,週刊少年チャンピオン,45410
2,週刊少年マガジン,45293
3,週刊少年ジャンプ,43010


上記は`mcname`ごとの行数を表します．それぞれ同一期間で集計しましたが，掲載作品数の違いが生じています．

### `miname`（雑誌巻号名）

ユニークな`miname`数を集計します．

In [76]:
df['miname'].nunique()

9215

`mcname`（雑誌）ごとに集計した`miname`（雑誌巻号）は：

In [150]:
df.groupby('mcname')['miname'].nunique().reset_index()

,mcname,miname
0,週刊少年サンデー,2303
1,週刊少年ジャンプ,2299
2,週刊少年チャンピオン,2313
3,週刊少年マガジン,2300


ほぼ同数ですが，雑誌によって微妙に巻号数が異なることがわかります．

次は，`miname`（雑誌巻号）ごとに`mcname`（マンガ作品）数を集計してみます．

In [78]:
df.groupby('miname')['cname'].nunique().sort_values().reset_index()

,miname,cname
0,週刊少年マガジン 1975年 表示号数1,8
1,週刊少年マガジン 1971年 表示号数1,8
2,週刊少年マガジン 1974年 表示号数45,9
3,週刊少年サンデー 1973年 表示号数5,9
4,週刊少年サンデー 1974年 表示号数33,9
...,...,...
9210,週刊少年マガジン 2010年 表示号数7,35
9211,週刊少年ジャンプ 2014年 表示号数4,37
9212,週刊少年チャンピオン 2004年 表示号数1,37
9213,週刊少年ジャンプ 2003年 表示号数37,40


1970年付近の黎明期は掲載数が少なめだったようです．また，雑誌によってタイミングはバラバラですが，非常に沢山のマンガを掲載する**外れ値**が存在することもわかります．

### `cname`（マンガ作品名）

ユニークな`cname`（マンガ作品）数を集計します．

In [79]:
df['cname'].nunique()

6830

`mcname`（雑誌）ごとに`cname`（マンガ作品）数を集計します．

In [151]:
df.groupby('mcname')['cname'].nunique().reset_index()

,mcname,cname
0,週刊少年サンデー,1485
1,週刊少年ジャンプ,2105
2,週刊少年チャンピオン,1713
3,週刊少年マガジン,1558


ジャンプが圧倒的に多いですね…．試しに`cname`（マンガ作品）ごとに掲載数を集計してみます．

In [81]:
df_tmp = df[['mcname', 'cname']].value_counts().reset_index()
df_tmp.columns = ['mcname', 'cname', 'counts']
df_tmp

,mcname,cname,counts
0,週刊少年ジャンプ,こちら葛飾区亀有公園前派出所,1968
1,週刊少年マガジン,はじめの一歩,1184
2,週刊少年サンデー,名探偵コナン,1008
3,週刊少年ジャンプ,ONE PIECE,890
4,週刊少年サンデー,MAJOR,748
...,...,...,...
6856,週刊少年ジャンプ,大衆娯楽漫画きくらげ劇場,1
6857,週刊少年ジャンプ,大攻防戦,1
6858,週刊少年ジャンプ,大山岳,1
6859,週刊少年ジャンプ,大宮ジェット,1


```{margin} あれ，あの長編作品は…？
シーズンごとに作品名が変わっているシリーズ作品（ドカベン，刃牙，浦鉄，ジョジョ等）は，それぞれ別作品として集計されていることにご注意ください．
```

連載期間が長いものを見てみましょう．こち亀，はじめの一歩，名探偵コナン，ONEPIECE，MAJORと各雑誌のレジェンドが連なります．
一方で，連載期間が短いものの中には，企画ものや読み切りが存在するようです．

雑誌ごとに，マンガ作品の連載期間に関して基礎集計します．

In [82]:
df_tmp.groupby('mcname')['counts'].describe()

,count,mean,std,min,25%,50%,75%,max
mcname,,,,,,,,
週刊少年サンデー,1485.0,31.077441,73.995156,1.0,1.0,2.0,27.0,1008.0
週刊少年ジャンプ,2105.0,20.431829,75.075635,1.0,1.0,1.0,11.0,1968.0
週刊少年チャンピオン,1713.0,26.508465,60.133614,1.0,1.0,3.0,24.0,635.0
週刊少年マガジン,1558.0,29.068678,75.465729,1.0,1.0,2.0,21.0,1184.0


やはりジャンプの平均連載期間が，他誌と比べて短いことがわかります．

### `epname`（各話タイトル）

ユニークな`epname`（各話タイトル）数を集計します．

In [85]:
df['epname'].nunique()

150241

意外と重複しているようです．集計してみます．

In [87]:
df['epname'].value_counts().reset_index()

,index,epname
0,プロ野球編,186
1,後編,134
2,前編,118
3,最終回,82
4,第三部 孤狼青春編,81
...,...,...
150236,＄33 カミナリ、おそるべし,1
150237,星に願いを・・・・ ／ 笑っていいですよ ／ 朝食,1
150238,〔□67〕時間目 日替わり勉強!!,1
150239,C.C.S.9/ 亀裂,1


プロ野球編ってことは…．

In [88]:
df[df['epname']=='プロ野球編']['cname'].value_counts().reset_index()

,index,cname
0,ドカベン,186


やっぱりドカベンですね！ドカベンってもしかして**〇〇編**の粒度でしかタイトルをつけないのでしょうか…？ドカベンの`epname`を集計してみます．

In [92]:
df[df['cname']=='ドカベン']['epname'].value_counts().reset_index()

,index,epname
0,プロ野球編,186
1,新潟明訓対神奈川明訓,1
2,外伝 不知火・土門伝説,1
3,殿馬一人外伝 秘打「白鳥の湖」誕生秘話,1


In [93]:
df[df['cname']=='ドカベン']['epname'].isna().sum()

446

プロ野球編以外のドカベンの各話タイトルは欠測しているため，これ以上のことはわからなそうです．

### `creator`（作者）

`df`に存在する`creator`（作者）数を集計します．

In [96]:
df['creator'].nunique()

2740

合計作品数が多い`creator`を調べてみます．

In [98]:
df['creator'].value_counts().reset_index().head(10)

,index,creator
0,水島新司,2968
1,秋本治,1876
2,高橋留美子,1735
3,浜岡賢次,1381
4,青山剛昌,1285
5,あだち充,1270
6,森川ジョージ,1224
7,満田拓也,1222
8,板垣恵介,1171
9,藤田和日郎,1101


メンツが強すぎます…．
個人的にはこち亀の`秋本治`先生が一番かなと予想していましたが，`水島新司`先生が圧倒的でした．

ちなみに，こち亀の`creator`を集計すると以下のようになります．

In [103]:
df[df['cname']=='こちら葛飾区亀有公園前派出所']['creator'].value_counts().reset_index()

,index,creator
0,秋本治,1866
1,山止たつひこ,101
2,秋本治ととみさわちなつ,1


`秋本治`先生は，デビュー時`山止たつひこ`というペンネームを使っていました．

（この101話分を足しても全然追いつかない`水島新司`先生がすごすぎますが…）

### `pageStart`（開始ページ）

`pageStart`（開始ページ）について`describe()`で基礎集計すると，以下のようになります．

In [111]:
df['pageStart'].describe().reset_index()

,index,pageStart
0,count,179866.000000
1,mean,210.909330
2,std,126.082707
3,min,1.000000
4,25%,107.000000
5,50%,205.000000
6,75%,305.000000
7,max,9999.000000


異常に大きいものがあることがわかります．
試しに`pageStart`が`numberOfPage`より大きいものを抜き出すと：

In [115]:
df[df['pageStart'] > df['numberOfPages']].sort_values('pageStart')[
    ['miname', 'cname', 'epname', 'pageStart', 'pageEnd', 
     'numberOfPages']]

,miname,cname,epname,pageStart,pageEnd,numberOfPages
56163,週刊少年マガジン 1989年 表示号数10,コータローまかりとおる!,NaN,49.0,67.0,36.0
56166,週刊少年マガジン 1989年 表示号数10,SHOGUN,第39話 将軍!!緒方晋作!!,69.0,88.0,36.0
56168,週刊少年マガジン 1989年 表示号数10,彼女はデリケート!,Lesson.7 コンピュータをたたきつぶせ,89.0,108.0,36.0
56169,週刊少年マガジン 1989年 表示号数10,THE STAR,NaN,109.0,128.0,36.0
56171,週刊少年マガジン 1989年 表示号数10,キラキラ!,NaN,129.0,148.0,36.0
...,...,...,...,...,...,...
120378,週刊少年マガジン 2004年 表示号数38,予告大作戦,乙,563.0,563.0,551.0
121886,週刊少年マガジン 2005年 表示号数2,もう、しませんから。,File.15 僕のジヒョンちゃんを紹介しますの巻,581.0,587.0,574.0
171052,週刊少年サンデー 2015年 表示号数35,戦争劇場,第48戦:手を出したら最後…,601.0,308.0,506.0
79256,週刊少年ジャンプ 1995年 表示号数1,ジョジョの奇妙な冒険,思い出したぞ!,2275.0,293.0,470.0


となります．最後の2つは明らかに`startPage`がおかしい気がするので，分析をすすめる際は注意が必要そうです．

### `pageEnd`（終了ページ）

`pageEnd`（終了ページ）について`describe()`で基礎集計すると，以下のようになります．

In [113]:
df['pageEnd'].describe().reset_index()

,index,pageEnd
0,count,179860.000000
1,mean,228.364308
2,std,122.080065
3,min,0.000000
4,25%,126.000000
5,50%,222.000000
6,75%,322.000000
7,max,1578.000000


`pageEnd`にも異常に大きいものが存在するようです．
試しに`pageEnd`が`numberOfPage`より大きいものを抜き出すと：

In [117]:
df[df['pageEnd'] > df['numberOfPages']].sort_values('pageEnd')[
    ['miname', 'cname', 'epname', 'pageStart', 'pageEnd', 
     'numberOfPages']]

,miname,cname,epname,pageStart,pageEnd,numberOfPages
56161,週刊少年マガジン 1989年 表示号数10,名門!第三野球部,第65話 緊張高まる決勝直前,29.0,48.0,36.0
56163,週刊少年マガジン 1989年 表示号数10,コータローまかりとおる!,NaN,49.0,67.0,36.0
56166,週刊少年マガジン 1989年 表示号数10,SHOGUN,第39話 将軍!!緒方晋作!!,69.0,88.0,36.0
56168,週刊少年マガジン 1989年 表示号数10,彼女はデリケート!,Lesson.7 コンピュータをたたきつぶせ,89.0,108.0,36.0
56169,週刊少年マガジン 1989年 表示号数10,THE STAR,NaN,109.0,128.0,36.0
...,...,...,...,...,...,...
120306,週刊少年マガジン 2004年 表示号数36,奇跡の少年,#24 止める,553.0,570.0,512.0
135578,週刊少年サンデー 2008年 表示号数4,ワイルドライフ,第231話:死領の罠,559.0,578.0,568.0
121886,週刊少年マガジン 2005年 表示号数2,もう、しませんから。,File.15 僕のジヒョンちゃんを紹介しますの巻,581.0,587.0,574.0
171029,週刊少年サンデー 2015年 表示号数35,アンペア,第9話 侵入者,283.0,600.0,506.0


最後の二つは明らかにおかしいことがわかります．

次に，`pageEnd`から`pageStart`を引いて，各話のページ数を算出してみます．

In [119]:
df_tmp = df.copy()
df_tmp['pages'] = df_tmp['pageEnd'] - df_tmp['pageStart']
df_tmp['pages'].describe().reset_index()

,index,pages
0,count,179860.000000
1,mean,17.453258
2,std,25.500113
3,min,-9969.000000
4,25%,16.000000
5,50%,18.000000
6,75%,19.000000
7,max,1439.000000


`pages`は0より大きいことが期待されるため，最小値は小さすぎます．また，最大値も非常識的な値となっています．

### `numberOfPages`（各号の合計ページ数）

`numberOfPages`（雑誌の合計ページ数）を`describe`で基礎集計します．
なお，`df`をそのまま`describe`してしまうと掲載作品数が多い雑誌巻号にバイアスのかかった統計量になってしまうため注意が必要です．
そこで，ここでは`miname`で中間集計した`df_tmp`を`describe`します．

In [135]:
df_tmp = df.groupby('miname')[
    ['numberOfPages', 'datePublished', 'price']].\
    first().reset_index()
df_tmp['numberOfPages'].describe().reset_index()

,index,numberOfPages
0,count,9202.000000
1,mean,402.709737
2,std,72.033356
3,min,36.000000
4,25%,344.000000
5,50%,420.000000
6,75%,458.000000
7,max,600.000000


最小値が小さすぎる気がします．
試しに`numberOfPages`でソートすると，

In [136]:
df_tmp.sort_values('numberOfPages').head(10)

,miname,numberOfPages,datePublished,price
7839,週刊少年マガジン 1989年 表示号数10,36.0,1989-02-22,180.0
6665,週刊少年チャンピオン 2012年 表示号数28,36.0,2012-06-07,257.0
5835,週刊少年チャンピオン 1995年 表示号数13,214.0,1995-03-09,200.0
2478,週刊少年ジャンプ 1974年 表示号数11,234.0,1974-03-11,130.0
7095,週刊少年マガジン 1974年 表示号数16,236.0,1974-04-14,130.0
7092,週刊少年マガジン 1974年 表示号数13,236.0,1974-03-24,130.0
7121,週刊少年マガジン 1974年 表示号数40,236.0,1974-09-29,130.0
7090,週刊少年マガジン 1974年 表示号数11,236.0,1974-03-10,130.0
2486,週刊少年ジャンプ 1974年 表示号数19,236.0,1974-05-06,130.0
7123,週刊少年マガジン 1974年 表示号数42,236.0,1974-10-13,130.0


最初の二つに関しては入力ミスが疑われます．降順にソートしてみます．

In [137]:
df_tmp.sort_values('numberOfPages', ascending=False).head(10)

,miname,numberOfPages,datePublished,price
2109,週刊少年サンデー 2013年 表示号数36,600.0,2013-08-07,257.0
8621,週刊少年マガジン 2005年 表示号数2,574.0,2005-01-12,238.0
8576,週刊少年マガジン 2004年 表示号数22,573.0,2004-05-19,229.0
8640,週刊少年マガジン 2005年 表示号数4,572.0,2005-01-19,238.0
3979,週刊少年ジャンプ 2004年 表示号数37,572.0,2004-08-23,229.0
8410,週刊少年マガジン 2000年 表示号数49,570.0,2000-11-15,230.0
1872,週刊少年サンデー 2008年 表示号数4,568.0,2008-01-14,248.0
2108,週刊少年サンデー 2013年 表示号数35,568.0,2013-07-31,257.0
2107,週刊少年サンデー 2013年 表示号数34,566.0,2013-07-24,257.0
1853,週刊少年サンデー 2008年 表示号数2,565.0,2008-01-07,238.0


特別号の可能性があるので，妥当性の判断が難しいです．

いずれにしても`numberOfPages`は欠測数が多いため，積極的に分析に利用する必要はなさそうに見えます．

In [138]:
df_tmp['numberOfPages'].isna().sum()

13

### `datePublished`（発行日）

`datePublished`（発行日）を`describe`で基礎集計します．
前述したように`df`を直接`describe`するとバイアスが乗るので，`miname`で中間集計した`df_tmp`に対して分析を実施します．

In [153]:
df_tmp = df.groupby('miname')[['datePublished']].\
    first().reset_index()
# 日付処理を容易にするため，`pd.to_datetime`で型変換
df_tmp['datePublished'] = pd.to_datetime(df_tmp['datePublished'])
df_tmp['datePublished'].describe().reset_index()

,index,datePublished
0,count,9215
1,unique,6362
2,top,1988-01-01 00:00:00
3,freq,4
4,first,1970-08-02 00:00:00
5,last,2017-07-06 00:00:00


次に，年単位で集計してみます．

In [154]:
df_tmp['yearPublished'] = df_tmp['datePublished'].dt.year
df_tmp.value_counts('yearPublished').reset_index().\
    sort_values('yearPublished', ignore_index=True)

,yearPublished,0
0,1970,84
1,1971,204
2,1972,204
3,1973,204
4,1974,199
5,1975,202
6,1976,200
7,1977,202
8,1978,202
9,1979,200


集計開始年（`1970`）および集計終了年（`2017`）以外は，年間およそ190-205回ほど発行していることがわかります．

### `price`（雑誌価格）

`price`（雑誌価格）を`describe`で基礎集計します．
前述したように`df`を直接`describe`するとバイアスが乗るので，`miname`で中間集計した`df_tmp`に対して分析を実施します．

In [155]:
df_tmp = df.groupby('miname')[['price']].\
    first().reset_index()

In [159]:
df_tmp['price'].describe().reset_index()

,index,price
0,count,9213.000000
1,mean,195.063606
2,std,44.769360
3,min,80.000000
4,25%,170.000000
5,50%,200.000000
6,75%,229.000000
7,max,371.000000


80円だった時代があったのでしょうか…？後ほど分析します．

### `publisher`（発行者）

`publisher`（発行者）に関して集計します． 前述したように`df`を直接集計するとバイアスが乗るので，minameで中間集計したdf_tmpに対して分析を実施します．

In [184]:
df_tmp = df.groupby('miname')[['mcname', 'publisher']].\
    first().reset_index()

In [185]:
df_tmp.groupby('mcname')['publisher'].\
    value_counts().reset_index(name='count')

,mcname,publisher,count
0,週刊少年サンデー,小学館 ∥ ショウガクカン,1715
1,週刊少年サンデー,小学館,222
2,週刊少年サンデー,"小学館 ∥ ショウガクカン,小学館",55
3,週刊少年ジャンプ,集英社 ∥ シュウエイシャ,1335
4,週刊少年ジャンプ,"集英社 ∥ シュウエイシャ,集英社",383
5,週刊少年ジャンプ,集英社,377
6,週刊少年チャンピオン,秋田書店 ∥ アキタショテン,1622
7,週刊少年チャンピオン,秋田書店 ∥ アキタ ショテン,338
8,週刊少年チャンピオン,秋田書店,56
9,週刊少年チャンピオン,"秋田書店 ∥ アキタショテン,秋田書店 ∥ アキタ ショテン",1


かなり表記がぶれているようです．

### `editor`（編集者）

In [182]:
df_tmp = df.groupby('miname')[['mcname', 'editor']].\
    first().reset_index()